# The fundarmental group of $\Sigma_{g,1}$ 

In [ ]:
require './freegroup.rb'

## symplectic generators

In [ ]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

In [ ]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

In [ ]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

## Random Word generator

In [ ]:
def word_generator(length: 1, genus: 3)
  rstr = ''
  length.times{|k| rstr += Rseq[0..(4*genus-1)].sample.to_s}
  return Word.new(rstr)
end
def wordgen(length: 1, genus: 3)
  word_generator(length: length, genus: genus)
end;nil

# Goldman bracket & Turaev cobracket

## Term class and Expansion class

In [ ]:
class Term < Hash
  '''
    A Hash with keys :coeff, :words and :divs
  '''
  def initialize(coeff: 0, words: [], divs: [])
    begin      
      self[:coeff] = coeff
      self[:words] = words
    rescue => e
      p e.backtrace
    end
  end
  
  def cyclic_reduce
    trm = self.class.new(coeff: self[:coeff], divs: self[:divs])
    trm[:words] = self[:words].map(&:cyclic_reduce)
    return trm
  end
  
  def show
    coeff = (self[:coeff] == 1) ? '' : "(#{self[:coeff]})"
    body = self[:words].map(&:show).join("\u{2227}")
    coeff + body
  end
  
  def equiv?(a_term)
    '''
      return 1(match), -1(reversely match) and 0(not match).
    '''
    (self[:words].size).times do |k|
      flags = self[:words].zip(a_term[:words].rotate(k)).map{|pair| pair[0].is_cyclically_same?(pair[1])}
      if flags.all?{|tf| tf}
        return (-1)**k
        break
      end
    end
    return 0
  end
end

#-----------------------------------
class Expansion < Array
  '''
    An Array of Terms
  '''
  def ==(another)
    raise ArgumentError, another.class unless another.is_a?(self.class)
    diff = self.concat( another*(-1) ).simplify
    words = diff[0][:words]
    return (words.empty? || words.include?(Word.new))
  end
  
  def *(int)
    raise ArgumentError, int.class unless int.is_a?(Integer)
    return self.each_with_object(self.class.new) do |term, expn|
      term[:coeff] *= int
      expn << term
    end
  end
  
  def wedge(a_Word)
    raise ArgumentError, a_Word.class unless a_Word.is_a?(Word)
    self.each_with_object(self.class.new) do |term, expn|
      term[:words] << a_Word
      expn << term
    end
  end
  
  def show(simplify_level=0)
    expn = case simplify_level
      when 1
        self.delete_if{|t| t[:coeff] == 0}
      when 2
        self.simplify
      else 
        self.rotate(0)
      end
    #---
    mstr = expn.map{|term| term.show}.join(' + ')
    return (mstr.empty?) ? '0' : mstr
  end

  def simplify
    expn = self.class.new
    monomials = self.map{|mono| mono.cyclic_reduce unless mono[:coeff] == 0}.compact
    until monomials.size == 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      #---
      monomials.each do |mono|
        switch = mono.equiv?(pop)
        unless switch == 0
          pop[:coeff] += mono[:coeff] * switch
          pop[:divs] << mono[:divs]
          mono[:coeff] = nil
        end
      end
      expn << pop if pop[:coeff] != 0
      monomials.delete_if{|mono| mono[:coeff].nil?}
    end
    return (expn.empty?) ? (self.class.new << Term.new) : expn
  end
end; nil

### scratch

In [ ]:
w = Word.new('abcde').cyclic_permutation
p w.class
nil

In [ ]:
terms = [
  Term.new,
  Term.new(words: [wordgen(8)], coeff: -1), 
  Term.new(words: [wordgen(3), wordgen(2)], coeff: 1)
]

terms.each{|t| printf t.show + ' |-reduce-> ' + t.cyclic_reduce.show + "\n" }
nil

In [ ]:
count = 10
count.times do |k|
  expn = Expansion.new() 
  5.times do |i|
    cf = ( (-1)**(i.modulo(3)) - (-1)**((i+1).modulo(3)) )/2
    ws = [3,3].map{|k| word_generator(k)}
    expn << Term.new(divs: '', coeff: cf, words: ws)
  end
  if expn.simplify.size < 4
    break
  end
end; nil

## Partition class

In [ ]:
class Partition
  def initialize(word: Word.new, index: 0)
    @word = word
    s = word.flatten.size
    @index = index.modulo(s)
    @cutends = [
      {index: (index - 1).modulo(s), letter: @word[index-1], sign: -1}, 
      {index: index, letter: @word[index], sign: 1}
    ]
  end
  attr_reader :index, :cutends
  
  def show()
    "#{@cutends[0][:letter]}|#{@cutends[1][:letter]}"
  end
  def term()
    "#{@cutends[1][:letter]} - #{@cutends[0][:letter].inverse}"
  end
end; nil

### scratch

In [ ]:
(-1).modulo(5)

In [ ]:
w = Word.new('abcde')
p w.show
w.size.times do |k|
  ptn = Partition.new(word: w, index: k)
  printf ptn.show + ' , ' + ptn.term + "\n"
end; nil

## Linking of a pair of Pairtitions

In [ ]:
def lk(ptn1, ptn2)
  '''
    Input: a pair of Partitions
    Output: -1 or 0 or 1
  '''
  [ptn1, ptn2].each{|ptn| raise ArgumentError, ptn.class unless ptn.is_a?(Partition)}
#   hashes = [ptn1, ptn2].map do |ptn|
#     w = ptn[:word].flatten
#     k = ptn[:div]
#     s = w.size
#     [
#       {index: ( k%s ) + 1, letter: w[ k%s ], sign: 1}, 
#       {index: k, letter: w[k-1].inverse, sign: -1}
#     ]
#   end
  ij_idxs = [0,1].product([0,1])
#  total = ij_idxs.map{|i,j| epsilon(hashes[0][i], hashes[1][j])}.sum
  total = ij_idxs.map{|i,j| epsilon( ptn1.cutends[i], ptn2.cutends[j] ) }.sum
  return (-1)*(total/2)
end

#--------------------------------------------
def epsilon(h1, h2)
  '''
    Input: Hashes {index: a pos. Integer, letter: a Letter, sign: 1 or -1}
    Output: -1 or 1
  '''
  output = h1[:sign]*h2[:sign]
  letters = [h1, h2].map do |h|
    (h[:sign] == 1) ? h[:letter] : h[:letter].inverse
  end
  output *= unless (letters[0] == letters[1])
    (Rseq.index(letters[0]) < Rseq.index(letters[1])) ? 1 : -1
  else
    (h1[:index] < h2[:index]) ? h2[:sign] : h1[:sign]*(-1)
  end
  return output
end; nil

## Goldman bracket $\nabla$

In [ ]:
def bracket(w1, w2)
  """
  retern: an Expansion (an Array of Terms {coeff: *, words: *, divs: *})
  """
  expn = Expansion.new
  unless (w1.show == '1' || w2.show == '1')
    ws = [w1, w2]
    (w1.size.times.to_a).product(w2.size.times.to_a).each do |inds|
      #--- the pair of partitions ---
      ptns = [0,1].map{|k| Partition.new(word: ws[k], index: inds[k])}
      #--- put the two words togather ---
      cpws = [0,1].map{|k| ws[k].cyclic_permutation(inds[k])}
      expn << Term.new(divs: inds, coeff: lk(*ptns), words: [cpws[0]*cpws[1]])
    end
  end
  return expn
end
#-----
def nabla(*args)
  input = if args[0].is_a?(Expansion)
            args[0]
          elsif args.size == 2 && args.all?{|w| w.is_a?(Word)}
            Expansion.new << Term.new(coeff: 1, words: args)
          else
            msg = "args.size = #{args.size}, args[0].class = #{args[0].class}"
            raise ArgumentError, msg
          end
  output = input.each_with_object(Expansion.new) do |term, expn|
    raise ArgumentError, term.inspect unless term[:words].size == 2
    expn.concat( bracket(*term[:words]) * term[:coeff] )
  end
  return output
end; nil

### scratch

In [ ]:
# w1 = Word.new(a[1], a[2], b[1].inverse)
# w2 = Word.new(b[2], a[1], a[1].inverse) #, b[2])
w1, w2 = wordgen(2), wordgen(3)
printf [w1, w2].map(&:show).join(', ')  + "\n-------\n"

bra = bracket(w1, w2)

bra.each do |t|
  t.each{|k,v| printf "#{k}: " + ((k == :words) ? v[0].show : v.to_s) + ", "}
  printf "\n---\n"
end
puts bra.show, "= " + bra.show(2)
nil

## Turaev cobraket $\delta$

In [ ]:
def cobracket(myw)
  """
  return: an Expansion (an Array of Terms {coeff: *, words: *, divs: *})
  """
  expn = Expansion.new
  unless myw.show == '1' 
    ptn_num_pairs = (myw.size.times.to_a).combination(2)  #all the pair of partition numbers
    ptn_num_pairs.each do |i,j|
      #--- the pair of Partitions ---
      ptns = [i,j].map{|k| Partition.new(word: myw, index: k)} 
      #--- divide the word --- 
      words = myw.cyclic_permutation(i).split(j-i).reverse
      expn << Term.new(divs: [i,j], coeff: lk(*ptns), words: words)
    end
  end
  #-----
  return expn
end
#-----
def delta(*args)
  input = case args[0]
          when Expansion
            args[0]
          when Word
            Expansion.new << Term.new(coeff: 1, words: [args[0]])
          else
            msg = "args.size = #{args.size}, args[0].class = #{args[0].class}"
            raise ArgumentError, msg
          end
  output = input.each_with_object(Expansion.new) do |term, expn|
    raise ArgumentError, term.inspect unless term[:words].size == 1
    expn.concat( cobracket(*term[:words]) * term[:coeff] )
  end
  return output
end; nil

### scratch

In [ ]:
#mw = Word.new(b[1], a[1], b[1]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
mw = wordgen(5)
printf mw.show + "\n-------\n"

cobra = cobracket(mw) #true) #

# cobra.each do |t|
#   t.each do |k,v| 
#     p "#{k}: " + ((k == :words) ? v[0].show : v.to_s)
#   end
#   printf "---\n"
# end

puts cobra.show, "= " + cobra.show(2)
nil

# Conditions

## Jacobi Identity for $\nabla$

$$\nabla\circ(\nabla\otimes 1)\circ N = 0$$

In [ ]:
three_words = Array.new(3)
max_length = 3

three_words.map!{|w| word_generator(rand(1..max_length))}
#--- display ---
puts three_words.map(&:show).join(" \u{2297} ") + "  |---> " + "\n------"
#---------------

total = Expansion.new
3.times do |k|
  triple = three_words.rotate(k)
  bra = bracket(*triple[0..1])
  #--- display ---
  puts "\t\u{2207}(#{triple[0..1].map(&:show).join(" \u{2297} ")}) \u{2297} #{triple[2].show} ="
  puts "\t\t+ (#{bra.show(1)}) \u{2297} #{triple[2].show}"
  #---------------
  brabra = nabla( bra.wedge(triple[2]) )
  total.concat( brabra )
end

#--- display ---
printf "---------\n" + total.show(1) + "\n"
#---------------
"=" + total.show(2)

## Involutivity for $\nabla$ and $\delta$

$$\nabla\circ \delta = 0$$

In [ ]:
length = 8
myw = word_generator(length) #Word.new('ffA') #

cobra = delta(myw)
total = nabla( cobra )

#--- display ---
puts "#{myw.show} |--\u{03B4}-->  " + cobra.show(1)
puts "-----\n |--\u{2207}--> " + total.show(1)
#---------------
"=" + total.show(2)

## Compatibility condition for $\nabla$ and $\delta$

$\forall v,w \in \pi$,
$$
\delta([v,w]) = v\cdot\delta(w)- w\cdot\delta(v),
$$
where 
$$
u\cdot(x\otimes y) := [u,x]\otimes y + x\otimes[u,y].
$$

**Note**: From the last formula, we immediately obtain the following:
$$
u \cdot(x\wedge y) = [u,x]\wedge y - [u,y]\wedge x.
$$
Futhermore, we may define $(x\wedge y)\cdot u$ as follows:
$$
(x\wedge y)\cdot u := x\wedge [u,y] - y \wedge [u,x],
$$
and have
$$
(x\wedge y)\cdot u = u \cdot(x\wedge y). 
$$

In [ ]:
v, w = word_generator(3), word_generator(5) #samples[:s2][0] #Word.new(a[1], a[2]),samples[:s4][0] #Word.new(b[1])
#--- display ---
puts "#{v.show} \u{2297} #{w.show} \n====="
#---------------

#--- Left-hand eq. -------------------
bra = bracket(v, w)
lhe = delta( bra )
#--- display ---
puts "[#{v.show}, #{w.show}] = " + bra.show(1) + "\n---"
printf "Left-hand Eq.:  " + lhe.show(2) + "\n====="
#---------------

#--- Right-hand eq. -------------------
rhe = Expansion.new
double_idx = [0,1].product([0,1])
rhe = double_idx.map do |i,j|
  pair01, sign01 = [v,w].rotate(i), (-1)**i
  #---
  u = pair01[0]
  cobra = cobracket(pair01[1]) * sign01 #; printf "#{u}.\u{03B4}(#{pair01[1].show}) = #{u}.(#{cobra.show(1)})\n---\n"
  cobra.map do |term|
    pair02, sign02 = term[:words].rotate(j), (-1)**j
    #---
    x, y = pair02[0], pair02[1]
    bra = (bracket(u, x) * (term[:coeff] * sign02))#; printf "\t[#{u}, #{x}] \u{2297} #{y} \n" + "\t=(#{bra.show(1)}) \u{2297} #{y}"
    bra.wedge(y)
  end.inject{|}
#   printf "\n---\n"
end
#-----
printf "\nRight-hand Eq.:  "+ rhe.show(2)

#-----------------------
lhe == rhe

An example whose results of the both side of the compatibility condition are very long.

In [ ]:
v,w = Word.new('fbC'), Word.new('CfDBe')
nil

# Experiments

## Cobracket calclation samples

In [ ]:
a1, b1 = Word.new(a[1]), Word.new(b[1])
c = Group.conjugate(b1, a1)

samples = {
  s0: [
    Word.new,
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^2,
    a1^3,
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse),
    a1*(comms[2]^2)*Group.commutator(comms[2].inverse,c)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2]),
    a1*(a1.inverse)*Word.new(b[1])*a1,
    c*(a1.inverse)*a1,
    c*(c.inverse),
    Word.new(a[1], b[1], b[1].inverse, b[2])
  ],
  s7: [
    a1*(Group.conjugate(a1, b1.inverse))*a1*(b1.inverse),
    a1*b1*((a1*(b1.inverse))^2),
    a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse),
    (a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse)).contract,
    a1*a1*(Word.new(b[2].inverse)^2)
    ]
  }

arr = samples #.slice(:s7) #:s0, :s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each{|s| puts s.show + "  |--\u{03B4}-->  " + cobracket(s).show(2) }
end;nil

## Well-definedness verifications

### bracket

In [ ]:
count = 500
count.times do |counter|
  words = [3,5].map{|k| wordgen(k)}
  bra1 = bracket(*words)
  bra2 = bracket(*(words.map(&:contract)))
  unless bra1 == bra2
    puts "#{words.map(&:show).join(' , ')}"
    puts "( #{bra1.show(1)} ) - ( #{bra2.show(1)} ) != (0)"
    break
  else
    printf "O" if counter.modulo(count/100) == 0
  end
  printf "\nPASS" if counter == count - 1
end; nil

In [ ]:
count = 500
count.times do |counter|
  words = [2,3].map{|k| wordgen(k)}
  bra1 = bracket(*words)
  bra2 = bracket(*(words.map{|w| Group.conjugate(w,wordgen(2))}))
  unless bra1 == bra2
    puts "#{words.map(&:show).join(' , ')}"
    printf "( #{bra1.show(1)} ) - ( #{bra2.show(1)} ) != (0)"
    break
  else
    printf "O" if counter.modulo(count/100) == 0
  end
  printf "\nPASS" if counter == count - 1
end; nil

### cobracket

In [ ]:
count = 500
count.times do |counter|
  word = wordgen(5)
  cobra1 = cobracket(word)
  cobra2 = cobracket(word.contract)
  unless cobra1 == cobra2
    printf "\n"
    puts word.show
    puts "( #{cobra1.show(1)} ) - ( #{cobra2.show(1)} ) != (0)"
    break
  else
    printf "O" if counter.modulo(count/100) == 0
  end
  printf "\nPASS" if counter == count - 1
end; nil

In [ ]:
count = 500
count.times do |counter|
  word = wordgen(3)
  cobra1 = cobracket(word)
  cobra2 = cobracket(Group.conjugate(word, wordgen(2)))
  unless cobra1 == cobra2
    printf "\n"
    puts word.show
    puts "( #{cobra1.show(1)} ) - ( #{cobra2.show(1)} ) != (0)"
    break
  else
    printf "O" if counter.modulo(count/100) == 0
  end
  printf "\nPASS" if counter == count -1
end; nil

## Condisions verifications

### Jacobi identity

In [ ]:
max_length = 5
count = 10 #500 #0 #
count.times do |k|
  three_words = Array.new(3).map!{|w| word_generator(rand(1..max_length))}
  total = [0,1,2].map do |k|
    triple = three_words.rotate(k)
    #------
    nabla( nabla(*triple[0..1]).wedge(triple[2]) )
    #------
  end.inject{|sum, e| sum.concat(e)}
  result = total.show(2)
  puts three_words.map(&:show).join(" \u{2297} ") + "  |--> " + result if result != '(0)'
end; nil

### Involutivity

In [ ]:
count = 100 #10000 #
length = 5 #10 #
count.times do |k|
  myw = word_generator(length)
  total = cobracket(myw).each_with_object(Expansion.new) do |h, expn|
    expn.concat( bracket(*h[:words]) * h[:coeff] )
  end
  result = total.show(2)
  p myw.show + " |--> " + result if result != "(0)"
end; nil

## The cobracket completely detects simpleness

## Degree-3 simpleness of $a_{1}[a_{2}, a_{3}]$

In [ ]:
x = a1*(Group.commutator(a[2], b[2])^2)
printf x.show + " |-\u{03b4}-> " + delta(x).show(2) + "\n"

count = 10
count.times do |counter|
  gamma = 3.times.map{|k| wordgen(length: rand(1..3), genus: 2)}.inject{|memo, w| Group.commutator(memo, w)}
  cobra_str = cobracket(x*gamma).show(2)
  printf "O" if counter.modulo(count/10) == 0
  if cobra_str == '(0)'
    puts "\n"
    p gamma.show, gamma.contract.show 
    break
  end
end; nil

In [ ]:
w = a1*Group.commutator(Group.commutator(a1, b1.inverse), a1)
p w.show

cobracket(w).show(2)

# TODO

* [ ] wedge の一方に 1 があるものの扱いをはっきりさせる。
* [ ] $x\wedge x$ をゼロと認識させる。
* [x] Expansion#show のオプションに "係数 0 の項だけ消去" を加えたい。
* [x] Compatibility が全然ダメ。
* [x] aaa などの Word について、動作が少し変。
* [x] Extension class のメソッドとして、係数 0 を消去したり、共通項で整理したりできるようにしたい。
* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。